In [23]:
import os, sys, time, random
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import cv2

# ========== PATHS ==========
PROJECT_ROOT = r"D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo"
SRC_DIR = os.path.join(PROJECT_ROOT, "3. src")
EXP_DIR = os.path.join(PROJECT_ROOT, "2. experiments")

DATA_ROOT = os.path.join(PROJECT_ROOT, "1.Dataset", "PWMFD_YOLO")  # sửa nếu tên khác
TRAIN_IMG_DIR = os.path.join(DATA_ROOT, "train", "images")
TRAIN_LAB_DIR = os.path.join(DATA_ROOT, "train", "labels")
VAL_IMG_DIR   = os.path.join(DATA_ROOT, "val", "images")
VAL_LAB_DIR   = os.path.join(DATA_ROOT, "val", "labels")

CKPT_DIR = os.path.join(PROJECT_ROOT, "4. Checkpoints", "yolov3")
LOG_DIR  = os.path.join(PROJECT_ROOT, "5. Results", "1. logs")

os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# ========== sys.path ==========
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

print("SRC_DIR:", SRC_DIR)
print("TRAIN_IMG_DIR:", TRAIN_IMG_DIR)
print("VAL_IMG_DIR  :", VAL_IMG_DIR)
print("CKPT_DIR     :", CKPT_DIR)
print("LOG_DIR      :", LOG_DIR)

# ========== DEVICE ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


SRC_DIR: D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\3. src
TRAIN_IMG_DIR: D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\1.Dataset\PWMFD_YOLO\train\images
VAL_IMG_DIR  : D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\1.Dataset\PWMFD_YOLO\val\images
CKPT_DIR     : D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\4. Checkpoints\yolov3
LOG_DIR      : D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\5. Results\1. logs
Device: cpu


In [24]:
import os

DATASET_ROOT = r"D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\1.Dataset"
YOLO_ROOT = os.path.join(DATASET_ROOT, "PWMFD_YOLO")

train_img_dir = os.path.join(YOLO_ROOT, "train", "images")
val_img_dir   = os.path.join(YOLO_ROOT, "val", "images")

train_txt = os.path.join(YOLO_ROOT, "train.txt")
val_txt   = os.path.join(YOLO_ROOT, "val.txt")

with open(train_txt, "w") as f:
    for name in sorted(os.listdir(train_img_dir)):
        if name.lower().endswith((".jpg", ".jpeg", ".png")):
            f.write(os.path.join(train_img_dir, name) + "\n")

with open(val_txt, "w") as f:
    for name in sorted(os.listdir(val_img_dir)):
        if name.lower().endswith((".jpg", ".jpeg", ".png")):
            f.write(os.path.join(val_img_dir, name) + "\n")

print("train.txt:", train_txt)
print("val.txt  :", val_txt)


train.txt: D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\1.Dataset\PWMFD_YOLO\train.txt
val.txt  : D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\1.Dataset\PWMFD_YOLO\val.txt


In [25]:
train_label_dir = os.path.join(YOLO_ROOT, "train", "labels")
val_label_dir   = os.path.join(YOLO_ROOT, "val", "labels")

n_train_img = len([f for f in os.listdir(train_img_dir) if f.lower().endswith((".jpg",".png",".jpeg"))])
n_train_lab = len([f for f in os.listdir(train_label_dir) if f.lower().endswith(".txt")])
n_val_img   = len([f for f in os.listdir(val_img_dir) if f.lower().endswith((".jpg",".png",".jpeg"))])
n_val_lab   = len([f for f in os.listdir(val_label_dir) if f.lower().endswith(".txt")])

print("TRAIN images:", n_train_img, "| labels:", n_train_lab)
print("VAL   images:", n_val_img,   "| labels:", n_val_lab)


TRAIN images: 5908 | labels: 5908
VAL   images: 1477 | labels: 1477


In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import os

IMG_SIZE = 416

class YoloDataset(Dataset):
    def __init__(self, img_dir, label_dir, img_size=416):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.img_size = img_size
        self.img_files = sorted([f for f in os.listdir(img_dir)
                                 if f.lower().endswith((".jpg",".jpeg",".png"))])

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_name = self.img_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.label_dir, os.path.splitext(img_name)[0] + ".txt")

        img = Image.open(img_path).convert("RGB").resize((self.img_size, self.img_size))
        img = np.array(img, dtype=np.float32) / 255.0
        img = np.transpose(img, (2,0,1))
        img = torch.from_numpy(img)

        targets = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f:
                    targets.append(list(map(float, line.split())))
        targets = torch.tensor(targets, dtype=torch.float32) if len(targets) else torch.zeros((0,5))

        return img, targets, img_name

def collate_fn(batch):
    imgs, targets, names = zip(*batch)
    imgs = torch.stack(imgs, 0)
    return imgs, list(targets), list(names)

train_ds = YoloDataset(train_img_dir, train_label_dir, IMG_SIZE)
val_ds   = YoloDataset(val_img_dir,   val_label_dir,   IMG_SIZE)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False, collate_fn=collate_fn)

# test 1 batch
imgs, targets, names = next(iter(train_loader))
print("Batch images:", imgs.shape)
print("Targets[0] shape:", targets[0].shape)
print("Sample name:", names[0])


Batch images: torch.Size([4, 3, 416, 416])
Targets[0] shape: torch.Size([1, 5])
Sample name: 002469.jpg


# train YOLOv3 baseline
Dùng dataset YOLO format đã tạo (train/images, train/labels, val/images, val/labels)
Import model/loss từ folder 3.src 
Lưu checkpoint vào 4.Checkpoints/yolov3/
Lưu log vào 5.Results/1.logs/

In [68]:
#Hyperparams + class map
NUM_CLASSES = 3
CLASS_NAMES = ["with_mask", "incorrect_mask", "without_mask"]

IMG_SIZE = 416
BATCH_SIZE = 8
EPOCHS = 1
LR = 1e-4
WEIGHT_DECAY = 5e-4

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


In [28]:
# ========== Dataset YOLO (đọc ảnh + label .txt) ==========
class YoloDetectionDataset(Dataset):
    """
    Read YOLO-format labels:
    each line: cls_id x_center y_center w h (normalized)
    """
    def __init__(self, img_dir, label_dir, img_size=416):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.img_size = img_size

        self.img_files = [f for f in os.listdir(img_dir)
                          if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        self.img_files.sort()

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_name = self.img_files[idx]
        img_path = os.path.join(self.img_dir, img_name)

        # read image (BGR -> RGB)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # resize
        img = cv2.resize(img, (self.img_size, self.img_size))

        # normalize [0,1], CHW
        img = img.astype(np.float32) / 255.0
        img = np.transpose(img, (2, 0, 1))
        img = torch.from_numpy(img)

        # read label
        label_path = os.path.join(self.label_dir, os.path.splitext(img_name)[0] + ".txt")
        targets = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    cls_id, xc, yc, w, h = map(float, line.strip().split())
                    targets.append([cls_id, xc, yc, w, h])

        if len(targets) == 0:
            targets = torch.zeros((0, 5), dtype=torch.float32)
        else:
            targets = torch.tensor(targets, dtype=torch.float32)

        return img, targets, img_name


def collate_fn(batch):
    imgs, targets, names = [], [], []
    for b_idx, (img, target, name) in enumerate(batch):
        imgs.append(img)
        names.append(name)

        if target.numel() > 0:
            # add batch index at column 0 => [batch_id, cls, xc, yc, w, h]
            t = torch.zeros((target.size(0), 6), dtype=torch.float32)
            t[:, 0] = b_idx
            t[:, 1:] = target
            targets.append(t)

    imgs = torch.stack(imgs, dim=0)
    targets = torch.cat(targets, dim=0) if len(targets) > 0 else torch.zeros((0, 6), dtype=torch.float32)
    return imgs, targets, names



In [29]:
train_ds = YoloDetectionDataset(TRAIN_IMG_DIR, TRAIN_LAB_DIR, img_size=IMG_SIZE)
val_ds   = YoloDetectionDataset(VAL_IMG_DIR,   VAL_LAB_DIR,   img_size=IMG_SIZE)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_fn)

print("Train samples:", len(train_ds))
print("Val samples  :", len(val_ds))


Train samples: 5908
Val samples  : 1477


In [30]:
import os, sys, glob

PROJECT_ROOT = r"D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo"

# In working dir hiện tại
print("Current working directory:", os.getcwd())

# Liệt kê các folder trong PROJECT_ROOT để xem tên chính xác
print("\nFolders in PROJECT_ROOT:")
for x in os.listdir(PROJECT_ROOT):
    if os.path.isdir(os.path.join(PROJECT_ROOT, x)):
        print(" -", x)

# Tìm folder src thật sự (có thể là '3. src' hoặc '3.src' hoặc '3_src')
candidates = [d for d in os.listdir(PROJECT_ROOT) if "src" in d.lower()]
print("\nSRC candidates:", candidates)

# Chọn SRC_DIR đúng (ưu tiên folder chứa file model)
SRC_DIR = None
for d in candidates:
    p = os.path.join(PROJECT_ROOT, d)
    if os.path.isdir(p) and (glob.glob(os.path.join(p, "*model*.py"))):
        SRC_DIR = p
        break

print("Detected SRC_DIR:", SRC_DIR)

# Add sys.path
if SRC_DIR and SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

print("sys.path[0]:", sys.path[0])

# In các file .py trong src
print("\nPython files in SRC_DIR:")
if SRC_DIR:
    for f in sorted(glob.glob(os.path.join(SRC_DIR, "*.py"))):
        print(" -", os.path.basename(f))
else:
    print("Không tìm thấy folder src. ")


Current working directory: d:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\2. experiments

Folders in PROJECT_ROOT:
 - 0. Setup
 - 1.Dataset
 - 2. experiments
 - 3. src
 - 4. Checkpoints
 - 5. Results
 - 6. Webcam
 - train

SRC candidates: ['3. src']
Detected SRC_DIR: D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\3. src
sys.path[0]: c:\Users\kimli\anaconda3\envs\facemask_env\python310.zip

Python files in SRC_DIR:
 - a_dataset_utils.py
 - b_model_yolo.py
 - c_model_se_yolo.py
 - d_losses.py
 - f_metrics.py


In [45]:
import importlib
import b_model_yolo

importlib.reload(b_model_yolo)

print("Reloaded from:", b_model_yolo.__file__)
print("Has count_parameters:", hasattr(b_model_yolo, "count_parameters"))
print("Available:", [x for x in dir(b_model_yolo) if "count" in x.lower() or "yolo" in x.lower()])


Reloaded from: D:\10.CAO HOC\2.MON HOC TREN TRUONG\2. HK1 2526\4. Applications of Machine Learning in Data Analysis\3.Project FaceMask\Dataset_Facemask_Yolo\3. src\b_model_yolo.py
Has count_parameters: True
Available: ['YOLOv3', 'count_parameters']


In [58]:
from b_model_yolo import YOLOv3, count_parameters
from d_losses import yolo_loss
print("Imported b_model_yolo: YOLOv3 and helpers available")
print('YOLOv3 ->', YOLOv3)
print('yolo_loss ->', yolo_loss.__name__)


Imported b_model_yolo: YOLOv3 and helpers available
YOLOv3 -> <class 'b_model_yolo.YOLOv3'>
yolo_loss -> yolo_loss


In [59]:
# Safe parameter inspection: ensure `model` is an instance, not the module

import types, torch

def _is_module_instance(obj):
    return isinstance(obj, torch.nn.Module)

try:
    if 'model' in globals() and _is_module_instance(globals()['model']):
        m = globals()['model']
        print("Using existing `model` instance for parameter check.")
    else:
        # Try to import and instantiate a temporary model for inspection
        from b_model_yolo import YOLOv3, count_parameters
        print("`model` not present or not an nn.Module; creating a temporary YOLOv3 for inspection.")
        m = YOLOv3(num_classes=NUM_CLASSES, img_size=IMG_SIZE)

    params = list(m.parameters())
    print("Number of param tensors:", len(params))

    if len(params) > 0:
        total = sum(p.numel() for p in params)
        trainable = sum(p.numel() for p in params if p.requires_grad)
        print("Total params:", total)
        print("Trainable params:", trainable)
    else:
        print("❌ Model has ZERO parameters -> model_yolo.py may be registering layers incorrectly.")

except Exception as e:
    print('Error during parameter inspection:', type(e).__name__, str(e))


Using existing `model` instance for parameter check.
Number of param tensors: 33
Total params: 4726504
Trainable params: 4726504


In [60]:
from b_model_yolo import YOLOv3, count_parameters

# Instantiate and move to device
model = YOLOv3(num_classes=NUM_CLASSES, img_size=IMG_SIZE)
model = model.to(device)

# Quick sanity checks: parameter count and forward pass shapes
print('Device for model:', next(model.parameters()).device if len(list(model.parameters())) else device)
print('Trainable params:', count_parameters(model))

import torch
imgs = torch.randn(min(2, BATCH_SIZE), 3, IMG_SIZE, IMG_SIZE).to(device)
outs = model(imgs)
for i, o in enumerate(outs):
    print(f'out[{i}] shape =', o.shape)


Device for model: cpu
Trainable params: 4726504
out[0] shape = torch.Size([2, 24, 26, 26])
out[1] shape = torch.Size([2, 24, 52, 52])
out[2] shape = torch.Size([2, 24, 104, 104])
out[0] shape = torch.Size([2, 24, 26, 26])
out[1] shape = torch.Size([2, 24, 52, 52])
out[2] shape = torch.Size([2, 24, 104, 104])


In [61]:
NUM_CLASSES = 3
IMG_SIZE = 416
LR = 1e-4
WEIGHT_DECAY = 5e-4

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# (tuỳ chọn) scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [62]:
def train_one_epoch(epoch):
    model.train()
    total_loss = 0.0

    for step, (imgs, targets, names) in enumerate(train_loader, start=1):
        imgs = imgs.to(device)
        targets = targets.to(device)    

        optimizer.zero_grad()

        outputs = model(imgs)

        loss = yolo_loss(outputs, targets)   # có thể là yolo_loss(outputs, targets, IMG_SIZE, ...)
        # ------------------------------------------------

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if step % 10 == 0:
            print(f"Epoch {epoch} | step {step}/{len(train_loader)} | loss {loss.item():.4f}")

    return total_loss / max(1, len(train_loader))


In [63]:
# Validate (val loss)
@torch.no_grad()
def validate_one_epoch(epoch):
    model.eval()
    total_loss = 0.0

    for imgs, targets, names in val_loader:
        imgs = imgs.to(device)
        targets = targets.to(device)

        outputs = model(imgs)
        loss = yolo_loss(outputs, targets)  

        total_loss += loss.item()

    return total_loss / max(1, len(val_loader))


In [69]:
#Training loop + lưu checkpoint best/last + log CSV
import os, time
import pandas as pd

best_val_loss = float("inf")
history = []

for epoch in range(1, EPOCHS + 1):
    t0 = time.time()

    train_loss = float(train_one_epoch(epoch))
    val_loss   = float(validate_one_epoch(epoch))

    if scheduler is not None:
        scheduler.step()

    dt = time.time() - t0
    lr_now = float(optimizer.param_groups[0]["lr"])

    print(f"[Epoch {epoch}/{EPOCHS}] train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | lr={lr_now:.6f} | time={dt:.1f}s")

    history.append({
        "epoch": int(epoch),
        "train_loss": train_loss,
        "val_loss": val_loss,
        "lr": lr_now
    })

    # ===== SAVE LAST (SAFE) =====
    last_path = os.path.join(CKPT_DIR, "last.pt")
    torch.save({
        "epoch": int(epoch),
        "model_state_dict": model.state_dict(),
        "train_loss": train_loss,
        "val_loss": val_loss,
    }, last_path)

    # ===== SAVE BEST (SAFE) =====
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_path = os.path.join(CKPT_DIR, "best.pt")
        torch.save({
            "epoch": int(epoch),
            "model_state_dict": model.state_dict(),
            "train_loss": train_loss,
            "val_loss": val_loss,
        }, best_path)
        print(" Saved BEST to:", best_path)

# ===== SAVE LOGS =====
log_csv = os.path.join(LOG_DIR, "yolov3_baseline_log.csv")
pd.DataFrame(history).to_csv(log_csv, index=False)
print(" Saved log to:", log_csv)


Epoch 1 | step 10/739 | loss 0.0000
Epoch 1 | step 20/739 | loss 0.0000
Epoch 1 | step 20/739 | loss 0.0000
Epoch 1 | step 30/739 | loss 0.0000
Epoch 1 | step 30/739 | loss 0.0000
Epoch 1 | step 40/739 | loss 0.0000
Epoch 1 | step 40/739 | loss 0.0000
Epoch 1 | step 50/739 | loss 0.0000
Epoch 1 | step 50/739 | loss 0.0000
Epoch 1 | step 60/739 | loss 0.0000
Epoch 1 | step 60/739 | loss 0.0000
Epoch 1 | step 70/739 | loss 0.0000
Epoch 1 | step 70/739 | loss 0.0000
Epoch 1 | step 80/739 | loss 0.0000
Epoch 1 | step 80/739 | loss 0.0000
Epoch 1 | step 90/739 | loss 0.0000
Epoch 1 | step 90/739 | loss 0.0000
Epoch 1 | step 100/739 | loss 0.0000
Epoch 1 | step 100/739 | loss 0.0000
Epoch 1 | step 110/739 | loss 0.0000
Epoch 1 | step 110/739 | loss 0.0000
Epoch 1 | step 120/739 | loss 0.0000
Epoch 1 | step 120/739 | loss 0.0000
Epoch 1 | step 130/739 | loss 0.0000
Epoch 1 | step 130/739 | loss 0.0000
Epoch 1 | step 140/739 | loss 0.0000
Epoch 1 | step 140/739 | loss 0.0000
Epoch 1 | step 150